# 전처리

## install packages

In [ ]:
!pip install transformers

!pip install hanja

     |████████████████████████████████| 122kB 7.6MB/s 
     |████████████████████████████████| 266kB 36.2MB/s 
     |████████████████████████████████| 245kB 43.9MB/s 
  Created wheel for hanja: filename=hanja-0.13.3-cp37-none-any.whl size=128426 sha256=88116b0d93ca32537060a1bee5f51d6aa1a44c644bc7f5f49167cd8b099d592b
  Stored in directory: /root/.cache/pip/wheels/45/fc/c9/b7e7cb5c86935a1a99e2ad07f763728f8f17560e7b815a4b27
  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp37-cp37m-linux_x86_64.whl size=44117 sha256=00d3178c46979b00b5d59764350627ac60a4f4d6560d09b60f48fe1d7267a22d
  Stored in directory: /root/.cache/pip/wheels/d9/45/dd/65f0b38450c47cf7e5312883deb97d065e030c5cca0a365030
Successfully built hanja pyyaml
ERROR: datascience 0.10.6 has requirement coverage==3.7.1, but you'll have coverage 5.5 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: coveralls 0.5 has requirement coverage<3.999,>=3

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
#from konlpy.tag import Mecab
import re
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from transformers import TFBertForSequenceClassification, BertTokenizerFast
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import hanja
from hanja import hangul

In [ ]:
train = pd.read_csv('../train_data.csv',index_col=0).reset_index(drop=True)
test = pd.read_csv('../test_data.csv',index_col=0).reset_index(drop=True)

In [ ]:
def prepare_text(data):
    transfer = []
    for index, row in data.iterrows():
        result = hanja.translate(row['title'], 'substitution') # transfer hanja to korean
        con = re.sub(r"[^a-zA-Z0-9가-힣]"," ",result) # remove Special Characters
        con = con.replace('종합'," ").strip()
        transfer.append(con)
    return transfer

## prepare data

In [ ]:
train['title'] = prepare_text(train)
test['title'] = prepare_text(test)

In [ ]:
train['title'][0]

'인천 핀란드 항공기 결항 휴가철 여행객 분통'

In [ ]:
train.tail(20)

,title,topic_idx
45634,NBA 미네소타 뎅 영입 팀버울브스 아니고 팀버불스,5
45635,한국무용협회 예술대상에 박명숙 김근희 김운미 박재근,3
45636,기고 아인슈타인 지휘에 맞춰 블랙홀 듀엣 연주가 시작됐다,3
45637,1보 코로나19 확진자 총 1만156명 어제 하루 94명 증가,2
45638,일자민당 원로 헌법9조는 세계유산 개정 바늘귀만큼도 안돼,4
45639,민주일반연맹 비정규직 차별철폐 공동행동,2
45640,전국대학원생노조 경북대 실험실 폭발 피해자 끝까지 책임져야,2
45641,창원 시내버스 노조 파업 투표 가결 30일 운행 멈춘다종합,2
45642,장석주 시인 사랑은 새로운 이야기가 잉태되는 순간,3
45643,남친과 함께 잔혹하게 동거인 학대 20대 여성 구속영장,2


In [ ]:
test.head()

,title
0,유튜브 내달 2일까지 크리에이터 지원 공간 운영
1,어버이날 맑다가 흐려져 남부지방 옅은 황사
2,내년부터 국가RD 평가 때 논문건수는 반영 않는다
3,김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것
4,회색인간 작가 김동식 양심고백 등 새 소설집 2권 출간


## tokenize

In [ ]:
MAX_LEN = 45
BATCH_SIZE = 32
NUM_EPOCHS = 5

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('beomi/kcbert-base')

In [ ]:
def bert_tokenizer(sent, MAX_LEN):
    encoded_dict = tokenizer.encode_plus(
        text = sent,
        add_special_tokens = True,      # Add '[CLS]' and '[SEP]'
        max_length = MAX_LEN,           # Pad & truncate all sentences.
        pad_to_max_length = True,
        return_attention_mask = True,
    )
    
    input_id = encoded_dict['input_ids']
    attention_mask = encoded_dict['attention_mask'] # And its attention mask (simply differentiates padding from non-padding).
    token_type_id = encoded_dict['token_type_ids'] # differentiate two sentences
    
    return input_id, attention_mask, token_type_id

In [ ]:
def tokenize_bert(data):
    input_ids = []
    attention_masks = []
    token_type_ids = []

    for data_sent in tqdm(data["title"]):
        try:
            input_id, attention_mask, token_type_id = bert_tokenizer(data_sent, MAX_LEN)
          
            input_ids.append(input_id)
            attention_masks.append(attention_mask)
            token_type_ids.append(token_type_id)

        except Exception as e:
            print(e)
            print(data_sent)
            pass
    return np.array(input_ids,dtype=int), np.array(attention_masks,dtype=int), np.array(token_type_ids,dtype=int)

In [ ]:
train_input_ids, train_attention_masks, train_token_type_ids = tokenize_bert(train)
train_labels = np.asarray(train['topic_idx'],dtype=np.float32)

train_inputs = (train_input_ids, train_attention_masks, train_token_type_ids)

  0%|          | 0/45654 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2132: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 45654/45654 [00:07<00:00, 6148.93it/s]


In [ ]:
test_input_ids, test_attention_masks, test_token_type_ids = tokenize_bert(test)
test_inputs = (test_input_ids, test_attention_masks, test_token_type_ids)

  0%|          | 0/9131 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2132: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 9131/9131 [00:01<00:00, 7697.92it/s]


# Model build : 감성분석 미세조정

In [ ]:
# attention과 hidden layer에 dropout 적용 : 0.3
model = TFBertForSequenceClassification.from_pretrained('beomi/kcbert-base',from_pt=True, num_labels=7, attention_probs_dropout_prob=0.3, hidden_dropout_prob=0.3)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=2)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss=loss, metrics=[metric])

In [ ]:
model.fit(train_inputs, train_labels, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, validation_split = 0.2, callbacks=[earlystop_callback])

Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autogr

In [ ]:
results = model.predict(test_inputs)
topic = []
for i in range(len(results)):
    topic.append(np.argmax(results[i],axis=1))

submission = pd.read_csv('../data/sample_submission.csv')
submission['topic_idx'] = np.array(topic).T
submission.to_csv('../submission/bert_4.csv', index=False)

In [ ]:
results = model.predict(test_inputs)
np.save('../submission/bert4_save.npy',results['logits'])